## Load the SVHN Dataset

In [ ]:
import wget
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
train_url = "http://ufldl.stanford.edu/housenumbers/train_32x32.mat"
test_url = "http://ufldl.stanford.edu/housenumbers/test_32x32.mat"

train_data = wget.download(train_url)
test_data = wget.download(test_url)

In [ ]:
#type of origin file is matlab
data_train = loadmat(train_data)
data_test = loadmat(test_data)

## EDA

In [ ]:
#output is as a tuple
type(data_train), type(data_test)

In [ ]:
data_train.keys()

In [ ]:
data_test.keys()

In [ ]:
X_train = data_train['X']
X_test = data_test['X']

y_train = data_train['y']
y_test = data_train['y']

In [ ]:
type(X_train)

In [ ]:
X_train.shape

In [ ]:
def image_viz (X, y):
  fig, axes = plt.subplots(2,5,figsize=(12,6))

  #flatten the axes array for easy indexing
  axes = axes.ravel()

  for i in range(10):
      img = X[:, :, :, i] 
      
      axes[i].imshow(img)
      axes[i].set_title(f'Label: {y[i]}')
      axes[i].axis('off') #remove the thicks on axis

  plt.tight_layout()
  plt.show();

In [ ]:
image_viz(X_train, y_train)

In [ ]:
image_viz(X_test, y_test)

In [ ]:
#check y distribution(to see if the dataset is imbalance or not)
def label_dist(y):
    unique_labels, counts = np.unique(y,return_counts=True)
    return dict(zip(unique_labels, counts))

In [ ]:
train_dist = label_dist(y_train)
test_dist = label_dist(y_test)

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
plt.subplot(1, 2, 1)
plt.bar(train_dist.keys(), train_dist.values())
plt.title("train label dist")
plt.xlabel("Label")
plt.subplot(1, 2, 2)
plt.bar(test_dist.keys(), test_dist.values())
plt.title("test label dist")
plt.xlabel("Label")

plt.tight_layout()

In [ ]:
X_train.max(), X_train.min()

## Preprocessing

In [ ]:
## Normalization
def normalizer(x):
    x = x / 255
    return x

In [ ]:
x_train_norm = normalizer(X_train)
x_test_norm = normalizer(X_test)

In [ ]:
#output is 3(channel) matrix 32*32 in each other and have 73257 of this matrix 3 channel
np.shape(x_train_norm)

In [ ]:
# rgb to gray_scale(3 channel to 1 channel)
def convert_gray(x):
    return np.mean(x,axis=2,keepdims=True)

In [ ]:
x_train_norm_gray = convert_gray(x_train_norm)
x_test_norm_gray = convert_gray(x_test_norm)
np.shape(x_train_norm_gray)

In [ ]:
#reshape to prepare inputs for ML models
# change dim 4(or 3 becuase python is zero index) to dim 1(dimension 4 is then number of sample:73257 
    # that in a matrix m*n should be the first one)
# change dim 1(0) , 2(1) , 3(2) to dim 2 that means n in matrix(flatten)
def reshape(x):
    return x.transpose(3, 0, 1, 2).reshape(np.shape(x)[3],-1) # -1 means multiplication dim 0, 1, 2

In [ ]:
x_tr = reshape(x_train_norm_gray)
x_ts = reshape(x_test_norm_gray)

In [ ]:
x_tr_rgb = reshape(x_train_norm)
x_ts_rgb = reshape(x_test_norm)

In [ ]:
x_tr.shape

In [ ]:
x_tr_rgb.shape

In [ ]:
# train phase
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(max_depth=20, n_estimators=100)
rf_clf.fit(x_tr, y_train)

In [ ]:
# test phase
from sklearn.metrics import accuracy_score
y_pred = rf_clf.predict(x_ts)
score = accuracy_score(y_test,y_pred)

In [ ]:
y_test.shape

In [ ]:
print(y_pred.shape, type(y_pred))

In [ ]:
# dim reduction
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(x_tr)

In [ ]:
# transform
x_tr_pca = pca.transform(x_tr)
x_ts_pca = pca.transform(x_ts)

In [ ]:
# importance of each components
variances = pca.explained_variance_ratio_

In [ ]:
# cumulativepercentage of components' importance
cum_variances = np.cumsum(variances)

In [ ]:
plt.plot(cum_variances, marker = "o")
plt.grid()

In [ ]:
component_90 = np.argmax(cum_variances > 0.9) + 1

In [ ]:
pca_90 = PCA(n_components=component_90)
x_tr_90 = pca_90.fit_transform(x_tr)
x_ts_90 = pca_90.transform(x_ts)

In [ ]:
rf_clf = RandomForestClassifier(max_depth=20, n_estimators=100)
rf_clf.fit(x_tr_90, y_train)

In [ ]:
# test phase
y_pred = rf_clf.predict(x_ts_90)
score = accuracy_score(y_test,y_pred)

In [ ]:
# change 10 to 0
y_train_0 = np.where(y_train==10, 0, y_train)
y_test_0 = np.where(y_test==10, 0, y_test)

In [ ]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(n_estimators=200)
xgb_clf.fit(x_tr_90, y_train_0)

In [ ]:
y_pred = xgb_clf.predict(x_ts_90)
accuracy_score(y_test_0, y_pred)